In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from utils import get_spikes

In [2]:
def shanon_entropy(X, wlen):
    p = X.rolling(wlen).apply(lambda x: int(''.join(map(lambda x: str(int(x)), x)), 2)).value_counts()
    p /= p.sum()
    return -np.sum(p*np.log2(p))

In [13]:
def shanon_entropy_join(X, wlen):
    p = X.rolling(wlen).apply(lambda x: int(''.join(map(lambda x: str(int(x)), x)), 2))
    p = p.dropna()
    p["joined"] = p.values.tolist()
    p = p.joined.value_counts()
    p /= p.sum()
    return -np.sum(p*np.log2(p))

In [4]:
def resolution(X, n):
    return X.groupby(X.index // n).max()

In [11]:
df = pd.DataFrame({"x":[0,0,1,0,0,0,1,0,1,1,0,0,0],"y":[0,0,1,0,0,0,1,0,1,1,0,0,0]})
resolution(df,13)

,x,y
0,1,1


In [15]:
def mutual_info(X,cols,wlen):
    return shanon_entropy(X[[cols[0]]],wlen) + shanon_entropy(X[[cols[1]]],wlen) - shanon_entropy_join(X[cols],wlen)

In [40]:
mutual_info(df2,['x','y'],3)

1.7219280948873623

In [38]:
df2 = pd.DataFrame(columns=['x','y'])
df2['x']=np.zeros(12)
df2['y']=np.ones(12)
df2.loc[6:,'x'] = 1
df2.loc[6:,'y'] = 0
df2

,x,y
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
5,0.0,1.0
6,1.0,0.0
7,1.0,0.0
8,1.0,0.0
9,1.0,0.0


In [ ]:
mutual_info(df)